<div>
    <p style="float: right;"><img width="66%" src="templates/logo_fmriflows.gif"></p>
    <h1>Anatomical Preprocessing</h1>
    <p>This notebooks preprocesses anatomical MRI images by executing the following processing steps:

1. Reorient images to RAS
1. Crop FOV with FSL
1. N4-inhomogenity correction with ANTS
1. GM, WM and CSF segmentation with SPM
1. Brainmask creation and brain extraction with Nilearn
1. Normalization to ICBM template with ANTS</p>
</div>

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format:

    dataset
    ├── fmriflows_spec_preproc.json
    └── sub-{sub_id}
        └── anat
            └── sub-{sub_id}_{T1w_id}.nii.gz
            
**Note:** Subfolders for individual scan sessions are optional.

`fmriflows` will run the preprocessing on all files of a subject.

## Execution Specifications

This notebook will extract the relevant processing specifications from the `fmriflows_spec_preproc.json` file in the dataset folder.

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'fmriflows_spec_preproc.json')

with open(spec_file) as f:
    specs = json.load(f)

In [ ]:
# Extract parameters for anatomical preprocessing workflow
subject_list = specs['subject_list_anat']
session_list = specs['session_list_anat']
T1w_id = specs['T1w_id']
res_norm = specs['res_norm']
norm_accuracy = specs['norm_accuracy']
n_proc = specs['n_parallel_jobs']

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject identifiers
subject_list

In [ ]:
# List of session identifiers
session_list

In [ ]:
# Anatomical image identifier
T1w_id

In [ ]:
# Resolution of normalized images
res_norm

In [ ]:
# ANTs Normalization accuracy
norm_accuracy

In [ ]:
# Number of parallel jobs to run
n_proc

# Creating the Workflow

To ensure a good overview of the anatomical preprocessing, the workflow was divided into two subworkflows:

1. The Main Workflow, i.e. doing the actual preprocessing
2. Report Workflow, i.e. visualizating relevant steps for quality control

## Import Modules

In [ ]:
from os.path import join as opj
from nipype import Node, Workflow, Function, IdentityInterface
from nipype.interfaces.image import Reorient
from nipype.interfaces.fsl import RobustFOV
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration
from nipype.algorithms.misc import Gunzip
from nipype.interfaces.spm import NewSegment
from nipype.interfaces.io import SelectFiles, DataSink

In [ ]:
# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-r7219/spm12_mcr/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

In [ ]:
# Create fmriflows output folder if missing
import pathlib
pathlib.Path(opj(exp_dir, out_dir)).mkdir(parents=True, exist_ok=True) 

In [ ]:
# Creation of template brain with desired voxel resolution
template_dir = '/templates/mni_icbm152_nlin_asym_09c/'
brain_template = opj(template_dir, '1.0mm_brain.nii.gz')

In [ ]:
# Resample template brain to desired resolution
from nibabel import load, Nifti1Image
from nilearn.image import resample_img
from nibabel.spaces import vox2out_vox

img = load(brain_template)
target_shape, target_affine = vox2out_vox(img, voxel_sizes=res_norm)
img_resample = resample_img(img, target_affine, target_shape, clip=True)
norm_template = opj(template_dir, 'template_brain_%s.nii.gz' %'_'.join([str(n) for n in res_norm]))
img_resample.to_filename(norm_template)

## Create a subworkflow for the Main Workflow

### Implement Nodes

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Reduces FOV of images to remove lower head and neck
crop_FOV = Node(RobustFOV(output_type='NIFTI_GZ'), name='crop_FOV')

In [ ]:
# Corrects bias field
n4 = Node(N4BiasFieldCorrection(dimension=3), name='n4')

In [ ]:
# Gunzips images
gunzip = Node(Gunzip(), name='gunzip')

In [ ]:
# Segments brain into 5 classes (GM, WM, CSF, Skull & Head)
segment = Node(NewSegment(), name='segment')

In [ ]:
# Compute Brain Mask and Extract Brain
def get_brain_and_mask(in_file, segments):
    
    import nibabel as nb
    from nilearn.image import clean_img, mean_img, math_img
    from scipy.ndimage.morphology import (
        binary_fill_holes, binary_dilation, binary_erosion)
    from os.path import basename, abspath

    # Load T1w corrected image
    img = nb.load(in_file)

    # Brainmask is created from the probability tissue maps
    gm, wm, csf, skull, head = [s[0] for s in segments]
    img_gmwm = math_img("(img1 + img2) >= 0.25", img1=gm, img2=wm)
    img_csf = math_img("img1 >= 1.0", img1=csf)
    img_not_rest = math_img("(img1 + img2) >= 0.25", img1=head, img2=skull)
    img_mask = math_img("(img1 + img2 - img3) >= 1.0", img1=img_gmwm, img2=img_csf, img3=img_not_rest)

    # Improves brainmask by 1 x erosion, 2 x dilation & filling of wholes
    data_mask = binary_erosion(
                binary_fill_holes(
                binary_dilation(
                img_mask.get_data(),
                    iterations = 2)),
                    iterations = 1).astype('int8')
    img_mask = nb.Nifti1Image(data_mask, img.affine, img.header)

    # Extract Brain with Mask
    img_brain = math_img("img1 * img2", img1=img, img2=img_mask)

    # Store output in nifti files
    filename = abspath(basename(in_file))
    out_file = filename.replace('.nii', '_brain.nii')
    mask = filename.replace('.nii', '_brainmask.nii')
    img_brain.to_filename(out_file)
    img_mask.to_filename(mask)

    return out_file, mask

extract_brain = Node(Function(input_names=['in_file', 'segments'],
                              output_names=['out_file', 'mask'],
                              function=get_brain_and_mask),
                     name='extract_brain')

In [ ]:
# Compress segmentation files
def compress_segments(segments):

    import nibabel as nb
    from os.path import basename, abspath
    
    # Change the compression level of the NIfTI image
    nb.openers.Opener.default_compresslevel = 6

    # Go through the individual segments
    compressed_segments = []
    for s in segments:
        new_fname  = abspath(basename(s[0] + '.gz'))
        nb.load(s[0]).to_filename(new_fname)
        compressed_segments.append(new_fname)
    
    return compressed_segments

compressor = Node(Function(input_names=['segments'],
                           output_names=['compressed_segments'],
                           function=compress_segments),
                  name='compressor')

In [ ]:
# Specify parameters for ANTs normalization
if norm_accuracy == 'precise':
    par = {'metric': ['Mattes', 'Mattes', 'CC'],
           'radius_or_number_of_bins': [56, 56, 4],
           'transform_parameters': [[0.05], [0.08], [0.1, 3.0, 0.0]],
           'number_of_iterations': [[200, 100], [200, 100], [100, 70, 50, 20]],
           'sampling_strategy': ['Regular', 'Regular', 'None'],
           'sampling_percentage': [0.25, 0.25, 1.0],
           'smoothing_sigmas': [[2, 1], [1, 0], [3, 2, 1, 0]],
           'shrink_factors': [[2, 1], [2, 1], [8, 4, 2, 1]]
          }

elif norm_accuracy == 'fast':
    par = {'metric': ['Mattes', 'Mattes', 'Mattes'],
           'radius_or_number_of_bins': [32, 32, 56],
           'transform_parameters': [[0.01], [0.08], [0.1, 3.0, 0.0]],
           'number_of_iterations': [[1000], [500, 250, 100], [50, 20]],
           'sampling_strategy': ['Random', 'Regular', 'Regular'],
           'sampling_percentage': [ 0.15, 0.15, 0.25],
           'smoothing_sigmas': [[4], [4, 2, 0], [1, 0]],
           'shrink_factors': [[4], [4, 2, 1], [2, 1]]
          }

In [ ]:
# Normalize anatomy to ICBM template
antsreg = Node(Registration(fixed_image=norm_template,
                            num_threads=n_proc,
                            output_inverse_warped_image=True,
                            output_warped_image=True,
                            collapse_output_transforms=True,
                            dimension=3,
                            initial_moving_transform_com=True,
                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            write_composite_transform=True,

                            float=False,
                            interpolation='BSpline',
                            transforms=['Rigid', 'Affine', 'SyN'],
                            sigma_units=['vox'] * 3,
                            metric_weight=[1.0] * 3,
                            use_estimate_learning_rate_once=[True] * 3,
                            use_histogram_matching=True,

                            radius_or_number_of_bins=par['radius_or_number_of_bins'],
                            sampling_percentage=par['sampling_percentage'],
                            sampling_strategy=par['sampling_strategy'],
                            transform_parameters=par['transform_parameters'],
                            metric=par['metric'],
                            number_of_iterations=par['number_of_iterations'],
                            convergence_threshold=[1e-06, 1e-06, 1e-06],
                            convergence_window_size=[20, 20, 10],
                            smoothing_sigmas=par['smoothing_sigmas'],
                            shrink_factors=par['shrink_factors'],

                            verbose=True,
                            terminal_output='file'),
               name='antsreg')

### Create Main Workflow

In [ ]:
# Create main preprocessing workflow
mainflow = Workflow(name='mainflow')

In [ ]:
# Add nodes to workflow and connect them
mainflow.connect([(reorient, crop_FOV, [('out_file', 'in_file')]),
                  (crop_FOV, n4, [('out_roi', 'input_image')]),
                  (n4, gunzip, [('output_image', 'in_file')]),
                  (gunzip, segment, [('out_file', 'channel_files')]),
                  (segment, extract_brain, [('native_class_images', 'segments')]),
                  (segment, compressor, [('native_class_images', 'segments')]),
                  (n4, extract_brain, [('output_image', 'in_file')]),
                  (extract_brain, antsreg, [('out_file', 'moving_image')])
                  ])

## Create a subworkflow for the report Workflow

### Implement Nodes

In [ ]:
# Create visual figures for anatomical preprocessing
def plot_figures(sub, sess, n4, segments, brain, T1_template, norm_template, warped_file):
    
    import nibabel as nb
    from nilearn.plotting import plot_stat_map, plot_roi
    from nilearn.masking import apply_mask, unmask
    from matplotlib.pyplot import figure

    import numpy as np
    from nilearn.image import math_img, smooth_img
    from nilearn.plotting import find_cut_slices
    from os.path import basename, abspath
    
    title_txt = 'sub: %s' % sub
    
    # Add session suffix if present
    if sess:
        title_txt += ' - sess: %s' % sess
    
    # Visualize Tissue Segmentation of T1w
    img = nb.load(brain)
    data = np.stack((np.zeros(img.shape),
                     nb.load(segments[0][0]).get_data(),
                     nb.load(segments[1][0]).get_data(),
                     nb.load(segments[2][0]).get_data(),
                     nb.load(segments[3][0]).get_data(),
                     nb.load(segments[4][0]).get_data()), axis= -1)
    label_id = np.argmax(data, axis=-1)
    segmentation = nb.Nifti1Image(label_id, img.affine, img.header)

    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        
        cuts = find_cut_slices(segmentation, direction=e, n_cuts=12)[2:-2]
        plot_roi(segmentation, cmap='Accent', dim=1, annotate=False, bg_img=n4,
                 display_mode=e, title=title_txt + ' - %s-axis' % e,
                 resampling_interpolation='nearest', cut_coords=cuts, axes=ax)
    
    out_segmentation = basename(brain).replace('brain.nii.gz', 'segmentation.png')
    fig.savefig(out_segmentation, bbox_inches='tight', facecolor='black',
                frameon=True, dpi=300, transparent=False)

    # Visualize Brain Extraction of T1w
    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        cuts = find_cut_slices(brain, direction=e, n_cuts=12)[2:-2]
        plot_stat_map(brain, title=title_txt + ' - %s-axis' % e, colorbar=False,
                      threshold='auto', bg_img=n4, cmap='magma', display_mode=e,
                      resampling_interpolation='nearest', dim=-1,
                      cut_coords=cuts, annotate=False, axes=ax)

    out_brain = basename(brain).replace('.nii.gz', '.png')
    fig.savefig(out_brain, bbox_inches='tight', facecolor='black', frameon=True,
                dpi=300, transparent=False)
    
    # Visualize T1w to MNI registration and deformation differences

    ## Smooth images
    img_brain = smooth_img(norm_template, 2)
    img_warp = smooth_img(warped_file, 2)

    # Mask images
    img_mask = math_img('img!=0', img=norm_template)
    data_brain = apply_mask(img_brain, img_mask)
    data_warp = apply_mask(img_warp, img_mask)

    # Remove very small values
    data_brain[data_brain<=1e-1] = 0
    data_warp[data_warp<=1e-1] = 0

    # Find most present value in the upper data value histogram
    freq, val = np.histogram(data_brain,bins=64)
    divider_brain = val[32+np.argmax(freq[32:])]

    freq, val = np.histogram(data_warp,bins=64)
    divider_warp = val[32+np.argmax(freq[32:])]

    # 'Equalize' their histogram
    img_brain = unmask(data_brain / divider_brain, img_mask)
    img_warp = unmask(data_warp / divider_warp, img_mask)

    ## Compute difference between warped image and brain template
    img_dif = math_img('(img1 - img2) * 10', img1=img_brain, img2=img_warp)

    fig = figure(figsize=(16, 8))
    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        cuts = find_cut_slices(img_dif, direction=e, n_cuts=12)[2:-2]
        plot_stat_map(img_dif, title=title_txt + ' - %s-axis' % e, colorbar=False,
                        threshold=0, bg_img=T1_template, display_mode=e, alpha=0.8,
                        resampling_interpolation='nearest', annotate=False, 
                        cmap='Spectral', cut_coords=cuts, axes=ax)
    
    out_warp = basename(warped_file).replace('.nii.gz', '.png')
    fig.savefig(out_warp, bbox_inches='tight', facecolor='black', frameon=True,
                dpi=300, transparent=False)
    
    return abspath(out_segmentation), abspath(out_brain), abspath(out_warp), sub, sess
    
# Create Plotting Node
create_figures = Node(Function(input_names=['sub', 'sess', 'n4', 'segments', 'brain',
                                           'T1_template', 'norm_template', 'warped_file'],
                              output_names=['out_segmentation', 'out_brain', 'out_warp',
                                            'sub', 'sess'],
                              function=plot_figures),
                name='create_figures')
create_figures.inputs.norm_template = norm_template
create_figures.inputs.T1_template = brain_template.replace('brain', 'T1')

In [ ]:
# Write the HTML report
def write_report(sub, sess, brain):
    
    import os
    
    with open('/reports/report_template_preproc_anat.html', 'r') as report:
        txt = report.read()
        txt = txt.replace('sub-placeholder', 'sub-%s' % sub)
        
        # Add session suffix if present
        if sess:
            txt = txt.replace('ses-placeholder', 'ses-%s' % sess)
            filename = 'sub-%s_ses-%s.html' % (sub, sess)
        else:
            txt = txt.replace('ses-placeholder', '')
            txt = txt.replace('__', '_')
            filename = 'sub-%s.html' % sub

    report_file = os.path.join('/data', 'derivatives', 'fmriflows', filename)
    
    with open(report_file, 'w') as report:
        report.writelines(txt)

# Create Report Node
create_report = Node(Function(input_names=['sub', 'sess', 'brain'],
                              function=write_report),
                     name='create_report')

### Create report Workflow

In [ ]:
# Create report workflow
reportflow = Workflow(name='reportflow')

In [ ]:
# Add nodes to workflow and connect them
reportflow.connect([(create_figures, create_report, [('sub', 'sub'),
                                                     ('sess', 'sess'),
                                                     ('out_brain', 'brain')
                                                     ])
                    ])

## Specify Input & Output Stream

In [ ]:
# Iterate over subject and session id
info_source = Node(IdentityInterface(fields=['subject_id', 'session_id']),
                   name='info_source')

iter_list = [('subject_id', subject_list)]

if session_list:
    iter_list.append(('session_id', session_list))
else:
    info_source.inputs.session_id = ''

info_source.iterables = iter_list

In [ ]:
# Create path to input files
def create_file_path(subject_id, session_id, T1w_id):

    # Get all anatomical files
    from bids.layout import BIDSLayout
    layout = BIDSLayout('/data/')

    search_parameters = {'datatype': 'anat',
                         'return_type': 'file',
                         'suffix': T1w_id,
                         'subject': subject_id,
                         'extensions': 'nii.gz',
                        }
    if session_id:
        search_parameters['session'] = session_id

    return layout.get(**search_parameters)[0]

select_files = Node(Function(input_names=['subject_id', 'session_id',
                                          'layout', 'T1w_id'],
                             output_names=['anat'],
                             function=create_file_path),
                    name='select_files')
select_files.inputs.T1w_id = T1w_id

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [('_%s' % T1w_id, ''),
                 ('_ras', ''),
                 ('_ROI', ''),
                 ('_corrected', ''),
                 ('c1', 'seg_gm_'),
                 ('c2', 'seg_wm_'),
                 ('c3', 'seg_csf_'),
                 ('c4', 'seg_skull_'),
                 ('c5', 'seg_head_')
                 ]

for sub in subject_list:
    substitutions += [('sub-%s' % sub, '')]

for sess in session_list:
    substitutions += [('ses-%s' % sess, '')]

for sub in subject_list:
    substitutions += [('_subject_id_%s/' % (sub),
                       'sub-{0}/sub-{0}_'.format(sub))]
    substitutions += [('/sub-%s_.nii' % sub,
                       '/sub-%s_T1w_corrected.nii' % sub)]
    substitutions += [('/sub-%s__.nii' % sub,
                       '/sub-%s_T1w_corrected.nii' % sub)]
    for sess in session_list:
        substitutions += [('_session_id_{1}sub-{0}/sub-{0}'.format(sub, sess),
                           'sub-{0}/sub-{0}_ses-{1}'.format(sub, sess))]
        substitutions += [('/sub-%s_ses-%s__.nii' % (sub, sess),
                           '/sub-%s_ses-%s_T1w_corrected.nii' % (sub, sess))]

substitutions += [('___', '_'),
                  ('__', '_'),
                  ('_.nii', '.nii')
                 ]

datasink.inputs.substitutions = substitutions

## Create Anatomical Preprocessing Workflow

In [ ]:
# Create anatomical preprocessing workflow
preproc_anat = Workflow(name='preproc_anat')
preproc_anat.base_dir = work_dir

preproc_anat.connect([(info_source, select_files, [('subject_id', 'subject_id'),
                                                   ('session_id', 'session_id')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the main workflow
preproc_anat.connect([(select_files, mainflow, [('anat', 'reorient.in_file')]),
                      
                      (mainflow, datasink, [
                          ('n4.output_image', 'preproc_anat.@n4'),
                          ('compressor.compressed_segments', 'preproc_anat.@segment'),
                          ('extract_brain.out_file', 'preproc_anat.@brain'),
                          ('extract_brain.mask', 'preproc_anat.@mask'),
                          ('antsreg.warped_image', 'preproc_anat.@warped_image'),
                          ('antsreg.inverse_warped_image', 'preproc_anat.@inverse_warped_image'),
                          ('antsreg.composite_transform', 'preproc_anat.@transform'),
                          ('antsreg.inverse_composite_transform', 'preproc_anat.@inverse_transform')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the report workflow
preproc_anat.connect([(info_source, reportflow, [('subject_id', 'create_figures.sub'),
                                                 ('session_id', 'create_figures.sess')
                                                ]),
                      
                      (reportflow, datasink, [
                          ('create_figures.out_segmentation', 'preproc_anat.@vis_segmentation'),
                          ('create_figures.out_brain', 'preproc_anat.@vis_brain'),
                          ('create_figures.out_warp', 'preproc_anat.@vis_warp'),
                      ]),
                     ])

In [ ]:
# Connect main workflow with report workflow
preproc_anat.connect([(mainflow, reportflow, [
                        ('n4.output_image', 'create_figures.n4'),
                        ('segment.native_class_images', 'create_figures.segments'),
                        ('extract_brain.out_file', 'create_figures.brain'),
                        ('antsreg.warped_image', 'create_figures.warped_file')
                        ]),
                     ])

## Visualize Workflow

In [ ]:
# Create preproc_anat output graph
preproc_anat.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph in the notebook (NBVAL_SKIP)
from IPython.display import Image
Image(filename=opj(preproc_anat.base_dir, 'preproc_anat', 'graph.png'))

# Run Workflow

In [ ]:
# Run the workflow in sequential mode
res = preproc_anat.run('Linear')

In [ ]:
# Save workflow graph visualizations in datasink
preproc_anat.write_graph(graph2use='flat', format='png', simple_form=True)
preproc_anat.write_graph(graph2use='colored', format='png', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph.png'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph.png'))
copyfile(opj(preproc_anat.base_dir, 'preproc_anat', 'graph_detailed.png'),
         opj(exp_dir, out_dir, 'preproc_anat', 'graph_detailed.png'));

In [ ]:
# Save template brain in `preproc_anat` folder
import shutil
from os.path import basename
new_path = '/data/derivatives/fmriflows/preproc_anat/%s' % basename(
    norm_template)

shutil.move(norm_template, new_path)